# 정부혁신1번가 - 정부혁신 홈페이지
* 정부혁신 제안(목록) > 정부혁신국민포럼 

출처 : https://www.innogov.go.kr/ucms/ogp/sug/list.do?menuNo=300011

```
ㆍ다함께 잘 사는 나라, 늘 혁신하는 정부를 위한 의견이 있다면 자유롭게 제안해 주세요.
ㆍ내가 낸 정부혁신제안이 추천 30표를 받으면 토론이 진행됩니다.
ㆍ추천 30표를 받지 못한 경우에도 정부혁신국민포럼 운영위원단의 논의를 통해 중요성과 시급성이 높다면 토론으로 바로 이관될 수 있습니다.
ㆍ타인의 명예를 훼손하는 내용 혹은 허위사실, 특정집단에 대한 혐오 표현 등 부적절한 제안은 관리자에 의해 숨김 처리 혹은 삭제될 수 있습니다.
```

## 크롤링 전 확인 하기
* https://www.innogov.go.kr/robots.txt
* 저작권 확인하기

In [ ]:
# 필요한 도구를 불러온다.
# 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구
import pandas as pd
# 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러올 수 있습니다.
import requests
# request로 가져온 웹사이트의 html 태그를 파싱하는데 사용합니다.
from bs4 import BeautifulSoup as bs
# 랜덤숫자를 생성한다.
import random
import time
# 대량 데이터 처리시 진행 상황을 표시합니다.
from tqdm import tqdm, trange
# 정규표현식
import re

## 정부혁신 국민포럼 페이지 가져오기
* 전체 내용을 가져오기 위해 목록에 있는 href 태그 안에 있는 url을 가져옵니다.

In [ ]:
pnum = 1
year_month = 201906

base_url = f"https://www.innogov.go.kr/ucms/ogp/sug/list.do?pnum={pnum}&menuNo=300011&sugMonthTp={year_month}&orderKey=registDtDesc"
base_url

### URL이 있는 태그의 위치
* 브라우저의 Inspect 기능을 통해 링크에 해당되는 selector를 가져오기
* Copy -> Copy selector를 통해 해당 태그의 위치를 가져옴

In [ ]:
#content > div.suggestion_list > ul > li > dl > dt > p > a

### 함수로 구현하기 전에 미리 원하는 정보를 가져와서 출력해 봄

In [ ]:
response = requests.get( base_url )
response

if response.status_code == 200:
    html = bs(response.text, 'html.parser')
    # 소스코드를 제대로 가져왔는지 확인해 봅니다.
    tags = html.select('#content > div.suggestion_list > ul')[0].find_all('a')
    # print(tags)
    for tag in tags:
        print(tag['href'])

### 위에서 실습한 내용을 페이지 번호를 추가해 가면서 크롤링해 오는 함수를 작성

In [ ]:
# 연-월별 전체 제안 목록을 가져옵니다.
def get_suggestion_list(pnum):
    # print(f'year_month : {year_month}, pnum: {pnum}')
    base_url = f"https://www.innogov.go.kr/ucms/ogp/sug/list.do?pnum={pnum}&menuNo=300011&orderKey=registDtDesc"
    # base_url 페이지를 받아온다.
    response = requests.get( base_url )
    # status_code 가 200 OK 일 때 아래의 코드를 실행한다.
    if response.status_code == 200:
        # 웹페이지의 내용을 가져왔다면 뷰티풀숩을 통해 html 태그를 가져온다.
        html = bs(response.text, 'html.parser')
        # html 태그 내용 중 아래의 selector를 통해 목록에 있는 다른 게시글을 가져오도록 한다.
        tags = html.select('#content > div.suggestion_list > ul')[0].find_all('a')
        # tags 의 내용이 없다면 suggestion_list 를 반환한다.
        if not tags :
            return(suggestion_list)
        else :
            # tags 에서 다른 게시글을 가져와 suggestion_list 에 담아준다.
            for tag in tags:
                suggestion_list.append(tag['href'])
        pnum += 1
        # 페이지번호를 하나씩 더해서 재귀호출을 합니다.
        get_suggestion_list(pnum)
        print(pnum)
    else:
        return(suggestion_list)

In [ ]:
# 목록의 내용만 크롤링 해옵니다.
# 목록의 링크 값을 가져와 suggestion_list 리스트에 담아줍니다.
suggestion_list = []
pnum = 1

get_suggestion_list(pnum)
suggestion_list    

### 제대로 href 태그의 링크를 가져왔는지 확인하기 

In [ ]:
print(len(suggestion_list))

suggestion_list[:3]

## 게시물 내용 가져오기
* href 목록을 통해 게시물의 내용을 가져오기

In [ ]:
url = '/ucms/ogp/sug/view.do?menuNo=300011&sgId=204&pnum=1'
base_url = f"https://www.innogov.go.kr{url}"
response = requests.get( base_url )
if response.status_code == 200:
    html = bs(response.text, 'html.parser')
    #content > div.vveiw_box1 > dl > dt
    title = html.select(
            '#content > div.vveiw_box1 > dl > dt')[0].get_text(strip=True)
    print(title)

In [ ]:
desc = html.select(
            '#content > div.vveiw_box1 > div.vveiw_name > ul > li > dl > dd')
category = desc[0].get_text(strip=True)
print(desc[0].get_text(strip=True))
print(desc[1].get_text(strip=True))
print(desc[2].get_text(strip=True))
print(desc[3].get_text(strip=True))

In [ ]:
# id값을 저장해 주기 위해 미리 추출 해보기
u = 'view.do?menuNo=300011&sgId=150&pnum=1'
u.split('=')[2].split('&')[0]

In [ ]:
# 목록 리스트에 있는 url를 넘겨주면 내용을 크롤링 해옵니다.
def get_suggestion_content(url):
    article = []
    base_url = f"https://www.innogov.go.kr/{url}"
    response = requests.get( base_url )
    if response.status_code == 200:
        html = bs(response.text, 'html.parser')
        # 제목을 가져옵니다.
        title = html.select(
            '#content > div.vveiw_box1 > dl > dt')[0].get_text(strip=True)
        # dd 태그에 있는 내용을 가져와서 카테고리, 시작일, 마감일, 제안인 정보를 추출합니다.
        desc = html.select(
            '#content > div.vveiw_box1 > div.vveiw_name > ul > li > dl > dd')
        category = desc[0].get_text(strip=True)
        content = html.select(
            '#content > div.vveiw_box1 > div.vveiw_cont > div > pre')[0].get_text(strip=True)
        start = desc[1].get_text(strip=True)
        end = desc[2].get_text(strip=True)
        author = desc[3].get_text(strip=True)
        # 추천인원을 가져옵니다.
        vote = html.select('#counter')[0].get_text(strip=True)
        # 해당 게시물의 Id 값을 가져옵니다.
        sgId = url.split('=')[2].split('&')[0]
        
        article.append(sgId)
        article.append(title)
        article.append(category)
        article.append(content)
        article.append(start)
        article.append(end)
        article.append(vote)
        article.append(author)
        #print(article)
        
#         time.sleep(random.randint(1,2))
        return article

### 각 게시물의 세부내용 가져오기

In [ ]:
data = []
# tqdm 을 사용해서 어느정도 가져왔는지 확인하기
for i, suggestion in tqdm(enumerate(suggestion_list)):
    # suggestion_list 에 있는 각 게시물에 접근해서 카테고리, 시작일, 마감일, 제안인, 제목, 내용 등을 가져옵니다.
    s = get_suggestion_content(suggestion)
    data.append(s)

In [ ]:
column_names = ["sgId", "title", "category", "content", "start", "end", "vote", "author"]
column_names

In [ ]:
# data

In [ ]:
data = pd.DataFrame(data, columns=column_names)
data.head()

In [ ]:
data.tail()

In [ ]:
data["category"].value_counts()

In [ ]:
data.to_csv("data/suggestion.csv", index=False)

In [ ]:
pd.read_csv("data/suggestion.csv").head()

In [ ]:
data.shape